In [ ]:
# Проверка наличия файла CSV с данными по размерам авардов

try:
    f = open('receive_awards_by_hour.csv')
    f.close()
except FileNotFoundError:
    print('Файл данных не существует!')
    f = open('receive_awards_by_hour.csv', 'w+', encoding='utf-8')
    f.write('Дата по часу,Бенефициарские,Другое,Игровой бот dice.id,Каналы телеграм,Награды делегатам,Награды за посты в Readdle.me,Награды через Визонатор,Прямые награды,Самоапы,Самоапы viz-social-bot,Чаты телеграм,Чаты телеграм (сплит),Эирдропы в чатах телеграм,Номер последнего блока в часе')
    f.close()

In [ ]:
# Проверка наличия файла CSV с данными по количествам авардов

try:
    f = open('count_awards_by_hour.csv')
    f.close()
except FileNotFoundError:
    print('Файл данных не существует!')
    f = open('count_awards_by_hour.csv', 'w+', encoding='utf-8')
    f.write('Дата по часу,Бенефициарские,Другое,Игровой бот dice.id,Каналы телеграм,Награды делегатам,Награды за посты в Readdle.me,Награды через Визонатор,Прямые награды,Самоапы,Самоапы viz-social-bot,Чаты телеграм,Чаты телеграм (сплит),Эирдропы в чатах телеграм,Номер последнего блока в часе')
    f.close()

In [62]:
# Находим номер последнего блока и даты в файле данных
from datetime import datetime

f = open('receive_awards_by_hour.csv', 'r', encoding='utf-8')
line = f.readlines()[-1]
print(line.split(','))
try:
    last_block_in_data = int(line.split(',')[-1])
    last_date_in_data = datetime.strptime(line.split(',')[0], '%Y-%m-%d %H:%M:%S')
except:
    last_block_in_data = 0
    last_date_in_data = 0

print(last_date_in_data, last_block_in_data)

['2022-10-29 14:00:00', '0.121998', '8.058979', '3.0899520000000003', '11.335970999999999', '23.212833000000003', '0.0', '0.0', '2.718976', '0.172999', '17.667958', '0.519995', '244.651421', '0.0', '235456\n']
2022-10-29 14:00:00 235456


In [25]:
from tvizbase.api import Api

api = Api()
tx = api.get_ops_in_block(456656)
print(tx)

Starting Tvibase
[{'trx_id': '0000000000000000000000000000000000000000', 'block': 456656, 'trx_in_block': 65535, 'op_in_trx': 0, 'virtual_op': 1, 'timestamp': '2018-10-16T01:19:21', 'op': ['witness_reward', {'witness': 'lb', 'shares': '0.095955 SHARES'}]}]


In [3]:
# Функция парсера
import re
import json
from datetime import datetime

def get_number_of_block(block): #Получение номера блока.
    block=json.loads(block)
    return block['block']

def parse_tx(transaction): #Получение типа транзакции и размера награды в виде {Тип_награды : Размер_в_SHARES}
    tx_timestamp = datetime.fromisoformat(transaction['timestamp'])
    tx_shares = float(transaction['op'][1]['shares'][:-7])
    if transaction['op'][0] == 'receive_award':
        transaction_memo = transaction['op'][1]['memo']
        if re.match(r'telegram:\d', transaction_memo) != None:
            tx_type = 'Чаты телеграм'
        elif re.match(r'channel:@', transaction_memo) != None:     
            tx_type = 'Каналы телеграм'
        elif re.match(r'split:', transaction_memo) !=None:
            tx_type = 'Чаты телеграм (сплит)'
        elif re.match(r'chat:', transaction_memo) !=None:
            tx_type = 'Эирдропы в чатах телеграм'
        elif re.match(r'viz://@', transaction_memo) !=None:
            tx_type = 'Награды за посты в Readdle.me'
        elif transaction_memo in ['', 'award']:
            if transaction['op'][1]['initiator'] == transaction['op'][1]['receiver'] == 'viz-social-bot':
                tx_type = 'Самоапы viz-social-bot'
            elif transaction['op'][1]['initiator'] == transaction['op'][1]['receiver']:
                tx_type = 'Самоапы'
            else:
                tx_type = 'Прямые награды'
        elif re.match(r'Награда за пост: https://golos.io/@', transaction_memo) !=None:
            tx_type = 'Награды за посты в GOLOS.IO'
        elif transaction['op'][1]['receiver'] == 'social':
            tx_type = 'Награды через Визонатор'
        elif transaction['op'][1]['initiator'] == 'dice.id':
            tx_type = 'Игровой бот dice.id'
        else:
            tx_type = ('Другое')
    if transaction['op'][0] == 'benefactor_award':
        if transaction['op'][1]['initiator'] == 'dice.id':
            tx_type = 'Игровой бот dice.id'
        else:
            tx_type = 'Бенефициарские'
    if transaction['op'][0] == 'witness_reward':
            tx_type = 'Награды делегатам'
    return{tx_type:tx_shares}

In [63]:
import schedule

from datetime import datetime
import json
from tvizbase.api import Api
api = Api()


for item in range(0, 3):
    parse_to_memory(item)
# Запуск демона, который будет каждый час обновлять данные

# Обработка полученных данных

# Добавление данных в CSV файл

# Повтор

2018-09-29 10:23:27
2018-09-29 10:23:30


In [4]:
import datetime

scan_start_at = datetime.datetime(2020, 10, 10, 12, 34, 35)
print(scan_start_at.hour)

13


In [13]:
import datetime

# dt_now = datetime.datetime.now()
dt_now = datetime.datetime.strptime('2018-09-29 10:30:00', '%Y-%m-%d %H:%M:%S')
# dt_then = datetime.datetime.strptime('2018-09-29 10:00:00', '%Y-%m-%d %H:%M:%S')
# scan_interval = datetime.timedelta(hours=0.5)
# if dt_now-dt_then>=scan_interval:
#     print('Интервал больше 1 часа')
# else:
#     print('Интервал меньше 1 часа')
# print(dt_now + scan_interval)

print(str(dt_now))

2018-09-29 10:30:00


In [6]:
# Обработка из папки tx_raw

import os
import json
import datetime
import operations as ops

awards_in_interval = dict.fromkeys(ops.cohorts, 0) 
count_awards_in_interval = dict.fromkeys(ops.cohorts, 0)

f_count = len(os.listdir('tx_raw')) #Количество файлов в папке с файлами данных

with open ('tx_raw/tx' + str(f_count) + '.txt') as fp:
    last_tx_in_files = fp.readlines()[-1] #Последняя транзакция в файлах
    last_block_in_files = int(json.loads(last_tx_in_files)['block']) #Последний блок в файлах

last_receive_award = ops.get_last_date_and_blocknumber_in_file('1.csv')
last_receive_award_date = last_receive_award['Последняя сохраненная дата']
last_receive_award_block = last_receive_award['Последний сохраненный блок']

for filenum in (1, 2+1):
    with open ('tx_raw/tx' + str(f_count) + '.txt') as fp:
        tx_in_file = fp.readlines()[0]
        print(len(tx_in_file), type(tx_in_file))
        print(tx_in_file)

240 <class 'str'>
{"trx_id": "0000000000000000000000000000000000000000", "block": 42900057, "trx_in_block": 65535, "op_in_trx": 0, "virtual_op": 1, "timestamp": "2022-10-29T14:32:09", "op": ["witness_reward", {"witness": "t3", "shares": "0.138999 SHARES"}]}

240 <class 'str'>
{"trx_id": "0000000000000000000000000000000000000000", "block": 42900057, "trx_in_block": 65535, "op_in_trx": 0, "virtual_op": 1, "timestamp": "2022-10-29T14:32:09", "op": ["witness_reward", {"witness": "t3", "shares": "0.138999 SHARES"}]}

